# Tensorflow K Nearest Neighbors Iris example

## Useful Links
* [aymericdamien's github](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/)
* [nfmcclure's github]( https://github.com/nfmcclure/tensorflow_cookbook/blob/master/05_Nearest_Neighbor_Methods/02_Working_with_Nearest_Neighbors/02_nearest_neighbor.ipynb)

In [1]:
import numpy as np
import tensorflow as tf

from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection

from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

In [2]:
iris = np.genfromtxt('data/iris.txt', delimiter=None) # load the text file
Y = iris[:,-1]   # target values is the last column
X = iris[:,0:-1] # features are the other columns

In [3]:
Xtr, Xte, Ytr, Yte = model_selection.train_test_split(X, Y, test_size=0.2, random_state=31)

## Sklearn KNN

In [4]:
knn = KNeighborsClassifier().fit(Xtr, Ytr)
Yhat = knn.predict(Xte)
print(metrics.mean_squared_error(Yte, Yhat))

0.0333333333333


## Tensorflow KNN

In [5]:
# number of neighbors
K=5

# tf graph input
xtr = tf.placeholder('float', shape=[None, 4])
xte = tf.placeholder('float', shape=[4])

# calc L1 distances
distances = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)

# k smallest values and their indexes
k_vals, k_inds = tf.nn.top_k(tf.negative(distances), k=K)

# predictions for the k smallest distances
knn = tf.gather(Ytr, k_inds)

# init the variables
init = tf.global_variables_initializer()

In [6]:
accuracy = 0.
# launch graph
with tf.Session() as sess:
    sess.run(init)
    for i in range(len(Xte)):
        # k nearest neighbors at that test point
        neighbors = sess.run(knn, feed_dict={xtr: Xtr, xte: Xte[i, :]})
        pred = Counter(neighbors).most_common(1)[0][0]
        # print("Test", i, "Prediction:", pred, "True Class:", Yte[i])
        if pred == Yte[i]:
            accuracy += 1
            
    accuracy /= len(Xte)
    print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667
